In [9]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import Model
import cv2
import numpy as np

In [10]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
#dim = (img_rows,img_cols)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [11]:
#x_train = cv2.resize(x_train[:], dim, interpolation = cv2.INTER_AREA) 
#print(x_train.shape)  

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



# RESIZING

## size reduced because of memory error
x_train_res = np.zeros((6000, 128, 128, 3))
x_test_res = np.zeros((1000, 128, 128, 3))
y_train_res = np.zeros((6000,))
y_test_res = np.zeros((1000,))


# source, desired size, resampling using pixel area relation
for i in range(6000):
    #rand1 = np.random.randint(0, 60000)
    a = cv2.resize(x_train[i], (128,128), interpolation = cv2.INTER_AREA)  
    x_train_res[i] = np.dstack((a,a,a))
    #y_train_res[i] = y_train[rand1]

print(x_train_res.shape)

for k in range(1000):
    #rand2 = np.random.randint(0,10000)
    b = cv2.resize(x_test[k], (128,128), interpolation = cv2.INTER_AREA)
    x_test_res[k] = np.dstack((b,b,b))
    #y_test_res[k] = y_test[rand2]
    
print(x_test_res.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(6000, 128, 128, 3)
(1000, 128, 128, 3)


In [13]:
# convert class vectors to binary class matrices
y_train_res = tensorflow.keras.utils.to_categorical(y_train_res, num_classes)
y_test_res = tensorflow.keras.utils.to_categorical(y_test_res, num_classes)

# define the mobilenet model

In [14]:
#model = Sequential()
base_model = mobilenet.MobileNet(input_shape=(128,128,3), alpha=0.25, include_top=False)
x = base_model.output
x = Flatten()(x)
x = Dense(100, activation='softmax')(x)
preds = Dense(10, activation='softmax')(x) #final layer with softmax activation
model = Model(inputs=base_model.input, outputs=preds)


#model.add(Flatten())
#model.add(Dense(100, activation='softmax'))
#model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adadelta(),
              metrics=['accuracy']) #,include_top = False,alpha = 0.25,input_shape = (128,128))

model.summary()

model.fit(x_train_res, y_train_res,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_res, y_test_res))
score = model.evaluate(x_test_res, y_test_res, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 8)         216       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 8)         32        
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 8)         0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 8)         72        
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 8)         32    

Train on 6000 samples, validate on 1000 samples
Epoch 1/12
6000/6000 [==============================] - 153s 26ms/sample - loss: 2.3005 - acc: 0.1733 - val_loss: 2.3164 - val_acc: 0.0780
Epoch 2/12
6000/6000 [==============================] - 1379s 230ms/sample - loss: 2.2711 - acc: 0.3078 - val_loss: 2.2725 - val_acc: 0.2560
Epoch 3/12
6000/6000 [==============================] - 1266s 211ms/sample - loss: 2.2415 - acc: 0.4835 - val_loss: 2.2289 - val_acc: 0.5920
Epoch 4/12
6000/6000 [==============================] - 163s 27ms/sample - loss: 2.2113 - acc: 0.6563 - val_loss: 2.1827 - val_acc: 0.8360
Epoch 5/12
5376/6000 [=========================>....] - ETA: 18s - loss: 2.1842 - acc: 0.7850